# Demo

The purpose of this notebook is to demonstrate the functionality of `gliderflightOG1`.

The demo is organised to show

- Step 1: Regressing the Seaglider steady flight model for chosen parameters

- Step 2: Recalculating the glider flight using the flight model and chosen parameters



In [ ]:

import xarray as xr
from gliderflightOG1 import plotters, tools
import numpy as np
from gliderflightOG1 import seaglider  # Assuming correct project style


In [ ]:

# 1. Load your dataset
glider = xr.open_dataset('../data/sg014_20080214T220104_delayed.nc')
glider = tools.calc_w_meas(glider)

# 2. Rename and clean up fields as needed
glider = glider.rename({
    'VBD_CC': 'VBD',
    'divenum': 'DIVENUM',
})


In [ ]:

# 3. Add missing variables
# a) Create UPDN from PITCH
glider = glider.assign_coords(TIME=glider['TIME'])  # Make sure TIME is coordinate
glider['UPDN'] = xr.where(glider['PITCH'] > 0, 1, -1)

# b) Create VERTICAL_SPEED from dP/dt
# Assume TIME is in seconds or known units


# c) Create fake C_VBD
glider['C_VBD'] = xr.zeros_like(glider['VBD'])


In [ ]:

# d) Add starter attributes
glider.attrs.update({
    'hd_a': 0.0036,     # starting guess
    'hd_b': 0.0098,     # starting guess
    'hd_c': 0.0010,     # starting guess
    'vbdbias': 0.0,     # start assuming no bias
    'abs_compress': 4e-6, # reasonable guess
    'therm_expan': 2e-5,  # reasonable guess
    'rho0': 1025.0,     # typical seawater density
    'vbd_min_cnts': 0.0,
    'vbd_cnts_per_cc': 1.0,
    'temp_ref': 10.0,
    'volmax': 0.0,
    'mass': 52.0,  # Approx mass of glider [kg]
})

# 4. Subselect a few dives to test
dives_to_use = glider['DIVENUM'].values
ensmat = dives_to_use[:20]  # first 20 dives for testing


In [ ]:
glider

In [ ]:

# 5. Which parameters to optimize
whichpar = [1, 1, 1, 0, 0, 0]  # solve for hd_a, hd_b, vbdbias

# 6. Run the regression
regressout, allwrms = seaglider.regress_all_vec(
    whichpar=whichpar,
    glider=glider,
    whichone=10,  # Ramsey's method
    ensmat=ensmat,
    plotflag=False,
    unstdyflag=0,
)

def renormalize_params(params, factors, direction='normalize'):
    """Renormalize parameters with given factors.

    Parameters
    ----------
    params : np.ndarray
        The parameters to transform. Expected order is
        [hd_a, hd_b, vbdbias, abs_compress, therm_expan, hd_c].
    factors : list or np.ndarray
        The factors for renormalization.
    direction : str, optional
        'normalize' to divide by factors, 'denormalize' to multiply by factors.
        Default is 'normalize'.

    Returns
    -------
    np.ndarray
        Transformed parameters.

    Raises
    ------
    ValueError
        If the direction is not 'normalize' or 'denormalize'.

    """
    if direction == 'normalize':
        return params * factors
    elif direction == 'denormalize':
        return params / factors
    else:
        raise ValueError("Direction must be 'normalize' or 'denormalize'.")

# Define renormalization factors
factors = np.array([1e3, 1e3, 1, 1e6, 1e5, 1e5])

# Create a vector of params from regressout with the same size as factors
params = np.zeros_like(factors)
params[np.array(whichpar, dtype=bool)] = regressout
# Renormalize regressout
#regressout_normalized = renormalize_params(regressout, factors, direction='normalize')
regressout_norm = renormalize_params(params, factors, direction='denormalize')

print("Optimized parameters:", regressout_norm)
print("Final WRMS:", allwrms)



In [ ]:
regressout_norm

In [ ]:
# 7. Apply optimized parameters and run flight model
# Update glider attributes with optimized parameters
glider.attrs.update({
#    'hd_a': regressout_norm[0],
#    'hd_b': regressout_norm[1], 
    'vbdbias': regressout_norm[2],
    # Keep other parameters as they were
})

# Run flight model with optimized parameters
try:
    # Calculate flight velocities using the optimized parameters
    glider_with_model = seaglider.flightvec_ds(
        glider, 
        xl=1.0,  # typical glider length in meters
        hd_a=glider.attrs['hd_a'],
        hd_b=glider.attrs['hd_b'], 
        hd_c=glider.attrs['hd_c']
    )
    
    print("Flight model calculation completed successfully!")
    print(f"Added variables: {list(set(glider_with_model.data_vars) - set(glider.data_vars))}")
    
    # Simple plot showing model results
    import matplotlib.pyplot as plt
    
    # Plot a subset of the data for visualization
    time_subset = slice(0, 500)  # First 500 data points
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Plot vertical velocities
    if 'VERTICAL_SPEED' in glider_with_model:
        ax1.plot(glider_with_model['TIME'][time_subset], 
                glider_with_model['VERTICAL_SPEED'][time_subset], 
                'b-', label='Observed', alpha=0.7)
    
    if 'umag' in glider_with_model:
        ax1.plot(glider_with_model['TIME'][time_subset], 
                glider_with_model['umag'][time_subset], 
                'r-', label='Model (umag)', alpha=0.7)
    
    ax1.set_ylabel('Velocity (m/s)')
    ax1.set_title('Glider Flight Model Results')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot glide angle if available
    if 'thdeg' in glider_with_model:
        ax2.plot(glider_with_model['TIME'][time_subset], 
                glider_with_model['thdeg'][time_subset], 
                'g-', label='Glide Angle (degrees)', alpha=0.7)
        ax2.set_ylabel('Angle (degrees)')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
    
    ax2.set_xlabel('Time')
    
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Error running flight model: {e}")
    print("This may be due to missing variables or incompatible data format.")
    print("Check that all required variables are present in the dataset.")